In [1]:
#after matching via uprn, parentuprn and addresses, we want to try to match via polygon
# we also want to be able to take in files that have been matched by hand to addresses

In [2]:
import pandas as pd
import time
import numpy as np
import copy
import re

In [ ]:
nana_process_file = "/Users/christine/OneDrive - University College London/WP1_Address list compilation/13_stage_3_from_NanaWei/PA_batch1_London_202402_processed.csv"
london_data = pd.read_csv(nana_process_file)

In [140]:
no_match = pd.read_csv("London1_no_match_0503.csv")

In [141]:
no_match.head()

,UCL_ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,ward,site_name_clean,site_name_GLA,site_name_LPA,uprn_x,decision,status,application_date,decision_date,short_site_name_LPA]
0,2,P/0466/21/PRIOR,HARROW,Prior Approval,Prior Approval Office to Residential,Change Of Use From Office (Class B1 (New Class...,20.0,31,NaN,HA1 1BA,NaN,NaN,31 - 33 College Road\r\r\nHarrow\r\r\nHA1 1BA,"31 - 33 College Road, Harrow, HA1 1BA",NaN,Grant,NaN,2/4/21,NaN,31 - 33 college road
1,3,P/2181/21/PRIOR,HARROW,Prior Approval,Prior Approval Office to Residential,Change Of Use From Office (Class B1 (New Class...,10.0,380,NaN,HA3 8EE,NaN,NaN,380 Kenton Road\r\r\nHarrow\r\r\nHA3 8EE,"380 Kenton Road, Harrow, HA3 8EE",NaN,Grant,NaN,20/05/2021,NaN,380 kenton road
2,11,21/01573/PRIMA,BARKING & DAGENHAM,"Prior Approval: Change of use from Commercial,...",NaN,Application for prior approval: Proposed chang...,NaN,280,NaN,RM8 2HF,Mayesbrook,Panache,"Panache, 280 Lodge Avenue, Dagenham, Barking A...","Panache, 280 Lodge Avenue, Dagenham, Barking A...",NaN,Prior Approval Not Required,NaN,NaN,NaN,panache
3,15,21/3558,BRENT,Prior Approval,NaN,Prior approval for change of use of second flo...,NaN,37,NaN,NW10 8UN,NaN,Kassinga House,"Second Floor, Kassinga House, 37 Winchelsea Ro...","Second Floor, Kassinga House, 37 Winchelsea Ro...",NaN,Prior approval required and approved,NaN,16/11/2021,NaN,second floor
4,16,22/0186,BRENT,Prior Approval,NaN,Prior approval for change of use of vacant sho...,NaN,"9,11",NaN,NW10 1DY,NaN,NaN,"9-11 Burnley Road, London, NW10 1DY","9-11 Burnley Road, London, NW10 1DY",NaN,Prior approval is required and approved,NaN,17/03/2022,NaN,9-11 burnley road


In [142]:
test_data = no_match.iloc[:10]

In [161]:
# parse from site_name_LPA

#version1: test_data['parsed_street'] = test_data['site_name_LPA'].apply(lambda x: re.findall("[0-9]+-?[0-9]?+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", x))


test_data['parsed_street'] = test_data['site_name_LPA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", x) if '-' in x else 
                                                                                   re.findall("[0-9]+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", x))



/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_66682/1807502907.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['parsed_street'] = test_data['site_name_LPA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", x) if '-' in x else


In [153]:
#re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", "31-33 College Road, Harrow, HA1 1BA")

#re.findall("[0-9]+.\s(.+Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)", "31 College Road, Harrow, HA1 1BA")

    
    #(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens)

['College Road']

In [163]:
test_data['parsed_street'] = test_data['parsed_street'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)

#re.search(r"(?<=AAA).*?(?=ZZZ)", your_text).group(0)
#test_data.head(10)


/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_66682/3449484892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['parsed_street'] = test_data['parsed_street'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)


In [52]:


re.findall("([0-9]+\s\w+\s\w.)(Road|Lane|Avenue)", "33 Mount View Road London N4 4SS")


[]

In [ ]:
([0-9]+\s\w+.Lane) 
([0-9]+\s\w+.Avenue)